- Title: Tosser
- Date: 2018-10-14 15:05
- Category: Games
- Tags: reinforcement-learning, python, games, gym
- Author: Varun
- Sumary: How to solve an interesting question




# The Game

While discussing interview questions with a person (thanks Joe) I had met at a machine learning meet, the conversation invariably turned to how interview candidates always throw out buzzwords and jump into the question without thinking about it. We then asked each other our favourite interview questions and we both did exactly that and jumped into the problem. In order to defend my behaviour, I argued that the question was far more subtle than what Joe had suggested.

The game was this: You can bet as much money as you want on the outcome of a coin toss. If it's heads, you win 3x what you bet (plus whatever you bet) while if it's tails, you lose whatever you've bet. A more concrete example, you bet \\$100 on the toss, if it's heads you win \\$300 so your total amount is now \\$400, while if it's tails, you're left with \\$0. This is obviously a positive game, betting $x$ has an expected return of $x$. (A normal toss game has an expected return of 0 in comparison)

Note I define return as winnings. So starting with 100 and leaving with 1000 is a return of 900. I define value as sum of winnings + initial investment. So starting with 100 and leaving with 1000 is a value of 1000.

The subtlety of this game is introduced in the fact that you only get a finite number of turns with this game. 

# The Subtlety

If this game had only 1 turn, you'd bet as much money as possible on the game. After all, who'd give you a 50% chance at a 300% return, or an expected return of 100% (albeit slightly risky). If this game had infinite number of turns, I would play with the minimum bet each time as my expected return is the minimum bet and the chance of going to 0 is very very rare. Note, this is also why when, playing in a casino, you reduce the number of bets you make because each bet has a negative expected value. 

With a finite number of turns, the game changes. If I had 2 turns, betting all my money each time still seems sound since 25% of the time I'd end up 16x or 1500% return with a 25% probability - still appetizing. But nothing 75% of the time. For a game with 10 turns, I'd make a 1 million x return, but this is a 1 in 1024 chance of happening, everything else is 0. Note that my expected return is $(2^n-1)x$ using this strategy, but my median return is $-x$. As $n \rightarrow \infty$ my expected return also grows, but my chance of winning keeps decreasing. Now I don't play the lottery, and if a billionaire decides to match every dollar in the lottery with 3 of his own, I still wouldn't play the game. So obviously, this strategy of betting everything on each toss is not something I'd want to do



# Constant Betting Strategy

The first time I posed this question to another, the first response I got was what was I optimizing for? It's clear that optimizing for expected value results in a bet everything each toss, but this is not a good strategy as the number of tosses increase as the chance of any payout diminishes. The next response was to bet half my stash each time so I'd never be out of the game, so I thought I'd look at various statistics for a constant $\lambda$

Let $\lambda \in [0,1]$ be the percentage of our current stash $x$ to bet on the toss. Value $V$ after one toss

\begin{align}
V|Heads =& (1-\lambda) x + 4 \lambda x\\
 =& (1 +3 \lambda)x \\
V|Tails =& (1-\lambda) x 
\end{align}

Hence $E(V) = (1+\lambda)x$ 

Looking at 2 tosses

\begin{align}
V|HH =& (1 -\lambda)(V|H) + 4\lambda (V|H) \\
 =& (1 +3 \lambda)(V|H) \\
 =& (1 +3 \lambda)^2x \\
V|HT =& (1 +3 \lambda)(V|T)\\
=& (1 +3 \lambda)(1-\lambda) x
\end{align}

and 

\begin{align}
V|TT =& (1 -\lambda)(V|T) \\
 =& (1 - \lambda)^2x \\
V|TH =& (1 +3 \lambda)(V|T)\\
=& (1 +3 \lambda)(1-\lambda) x
\end{align}

This should give us some intuition on the distribution. For a particular fixed strategy, i.e. constant $\lambda$, we now have a distribution given number of heads, $h$ and number of tails $t$

\begin{align}
V|H=h, T=t &= (1+3\lambda)^h (1-\lambda)^t x
\end{align}

For $\lambda =1 $  we can see that even one tail results in value = 0, while $\lambda=0$ results in a value of x. This is clearly a binomial expansion, and this makes this quite easy to analyse, in that the more heads has strictly more profit for a fixed $h+t$.
Let's have a look at mean, median and 25%, 75% results in this space.

In [27]:
import numpy as np 
def bin_table(lam, n):
    """
    Returns array of (n+1) * 2 representing count, earnings
    """
    from scipy.special import comb
    
    hv = 1+3*lam
    tv = 1 - lam
    table = np.zeros((n+1, 2))
    for ntails in range(n+1):
        nheads = n - ntails
        table[ntails, 0]=  int(comb(n, nheads))
        table[ntails, 1] = hv**nheads * tv ** ntails
    return table
        

In [29]:
bin_table(0.5, 8)[:,1]

array([1.52587891e+03, 3.05175781e+02, 6.10351562e+01, 1.22070312e+01,
       2.44140625e+00, 4.88281250e-01, 9.76562500e-02, 1.95312500e-02,
       3.90625000e-03])